In [1]:
# import packages for data manipulation
import pandas as pd
import numpy as np

# import packages for interactive plots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets, interactive, Layout, HBox, VBox
import calendar
from IPython.display import display
from ipywidgets.embed import embed_minimal_html

In [2]:
data = pd.read_csv("../avocado.csv")

In [3]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

In [4]:
# Make a dropdown to select the year, or "All"
year = widgets.Dropdown(
    options=['All'] + list(data['year'].unique()),
    value='All',
    description='Year:',
)
# Make a dropdown to select the avocado type, or "All"
avocado_type = widgets.Dropdown(
    options=['All'] + list(data['type'].sort_values().unique()),
    value='All',
    description='Type:',
)

# Make a dropdown to select the Region, or "All"
region = widgets.Dropdown(
    options=['All'] + list(data['region'].sort_values().unique()),
    value='All',
    description='Region:',
)

def custom_interactive_plot(region,year,avocado_type):
    """
    Filters and plot the average price of avocado over time

    Args:
    -----
        * region (str): the US region to filter on, or "All" to display all regions

        * year (str): the year to filter on, or "All" to display all years
        
        * avocado_type (str): the avocado type to filter on, or "All" to display all types

        Note: the dataframe to plot is globally defined here as `data`

    Returns:
    --------
        A plotly line chart

    """
    
    df2 = data.copy()
    if year != 'All':
        df2 = df2[df2.year == year]
    if avocado_type != 'All':
        df2 = df2[df2.type == avocado_type]
    if region != 'All':
        df2 = df2[df2.region == region]
    if region == 'All':
        df2 = df2[df2.region == 'TotalUS']
    df2['mean'] = df2.AveragePrice.mean()
    df2['ltma'] = df2['AveragePrice'].rolling(window=52).mean()
    df2['stma'] = df2['AveragePrice'].rolling(window=12).mean()
    
    melt_df = pd.melt(df2[['Date','AveragePrice','mean','ltma','stma']],id_vars='Date')

    if len(df2)>0:
        #fig = px.line(df2, x='Date', y='AveragePrice')
        fig = px.line(melt_df,x='Date', y='value', color='variable')
        fig.update_layout(
            title_text=f"Average Price of Avocado over Time, Region: {region}, Year: {year}, Type {avocado_type}",
            xaxis_tickformat = '%b <br>%Y')
        fig.update_xaxes(nticks=24)
        fig.show()

    else:
        print(f"No data to show for {region} for avocado type {avocado_type} and year {year}")

In [5]:
interactive(custom_interactive_plot,region=region,year=year,avocado_type=avocado_type)

interactive(children=(Dropdown(description='Region:', options=('All', 'Albany', 'Atlanta', 'BaltimoreWashingto…

In [6]:
def custom_yoy_plot(region,avocado_type):
    """
    Filters and plot the average price of avocado over time

    Args:
    -----
        * region (str): the US region to filter on, or "All" to display all regions

        * year (str): the year to filter on, or "All" to display all years
        
        * avocado_type (str): the avocado type to filter on, or "All" to display all types

        Note: the dataframe to plot is globally defined here as `data`

    Returns:
    --------
        A plotly line chart

    """
    
    df2 = data.copy()
    if avocado_type != 'All':
        df2 = df2[df2.type == avocado_type]
    if region != 'All':
        df2 = df2[df2.region == region]
    if region == 'All':
        df2 = df2[df2.region == 'TotalUS']
    
    if len(df2)>0:
        df2['month'] = df2.Date.dt.month
        grp = df2.groupby(['region','month','year']).agg({'AveragePrice':'mean'}).reset_index()
        fig = px.line(grp, x='month', y='AveragePrice', color='year', render_mode='svg')
        fig.update_layout(title_text="Average Price of Avocado, year over year")
        fig.update_xaxes(nticks=12)
        fig.show()

    else:
        print(f"No data to show for {region} for avocado type {avocado_type}")

In [7]:
interactive(custom_yoy_plot,region=region,avocado_type=avocado_type)

interactive(children=(Dropdown(description='Region:', options=('All', 'Albany', 'Atlanta', 'BaltimoreWashingto…